# Partie 1

## Modules

In [ ]:
import openpyxl
from classes import Ressource,RessourceUnavailability,Task,TaskUnavailability
from read_input import read_ressources,read_ressources_unavailabilities,read_tasks,read_tasks_unavailabilities

from gurobipy import *

## Lecture des données

In [ ]:
ressources = read_ressources(1)
ressources_unavailabilities = read_ressources_unavailabilities(1)
tasks = read_tasks(1)
tasks_unavailabilities = read_tasks_unavailabilities(1)

## Structure de données

In [ ]:
L_ressources = [j for j in ressources]
L_tasks = [i for i in tasks]
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (float(task.latitude), float(task.longitude)) for task in tasks.values()}

# creer locations
# creer dists

# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html


## Modèle

In [ ]:
### Create Model
m = Model("trs0")

### Decision variables
# Task-ressource assignment
Task_ressource = {(t,r) : m.addVar(vtype = GRB.BINARY, name = f'x_{t}_{r}') for r in range(1, number_of_ressources + 1) for t in range(1, number_of_tasks + 1)}

# Time-task assignment
Start_task = {t : m.addVar(vtype = GRB.INTEGER, lb = 0, ub = 1440, name = f's_{t}') for t in range(1, number_of_tasks + 1)}
# Les s_t valent entre 0 et 1440 car les contraintes de temps (entre 8h et 18h) sont contenues dans les jeux de données.

# Est-ce que le travailleur se déplace entre les points a et b ?
Moving_worker = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(1, number_of_ressources + 1) for a in dict_pos.values() for b in dict_pos.values() if a != b}
# Technician cannot leave or return to a depot that is not its base
# print(Moving_worker)

